Importing the required Modules

In [283]:
import random
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
from keras.optimizers import SGD
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer
import pandas as pd


In [284]:
lemmatizer = WordNetLemmatizer()

Download the nltk packages

In [285]:
nltk.download('omw-1.4')
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

Init File of Intents.json

In [286]:
words = []
classes = []
documents = []
ignore_words = ["?", "!"]

read the intents.json file

In [287]:
data_file = open("../intents.json").read()
data_file

'{\n  "intent": [\n    {\n      "tag": "greeting",\n      "patterns": [\n        "Hi",\n        "How are you?",\n        "Hey there, how have you been?",\n        "Is anyone there?",\n        "Hello",\n        "Good day",\n        "Hiii",\n        "Hii"\n      ],\n      "responses": [\n        "Hi there, what can I do for you?",\n        "Hey there, how can I help?"\n      ]\n    },\n    {\n      "tag": "goodbye",\n      "patterns": ["Bye", "See you there", "Goodbye"],\n      "responses": [\n        "See you later, thanks for visiting ",\n        "Have a nice day",\n        "Bye! Come back again soon."\n      ]\n    },\n    {\n      "tag": "thanks",\n      "patterns": ["Thanks", "Thank you", "That\'s helpful", "Nice", "Good"],\n      "responses": ["Happy to help!", "Any time!", "My pleasure"]\n    },\n    {\n      "tag": "about",\n      "patterns": [\n        "about you",\n        "Who are you?",\n        "What are you?",\n        "Who you are?",\n        "what you can do",\n        "W

Converting the intents file from bytes to json 

In [288]:
intents=json.loads(data_file)
intents

{'intent': [{'tag': 'greeting',
   'patterns': ['Hi',
    'How are you?',
    'Hey there, how have you been?',
    'Is anyone there?',
    'Hello',
    'Good day',
    'Hiii',
    'Hii'],
   'responses': ['Hi there, what can I do for you?',
    'Hey there, how can I help?']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you there', 'Goodbye'],
   'responses': ['See you later, thanks for visiting ',
    'Have a nice day',
    'Bye! Come back again soon.']},
  {'tag': 'thanks',
   'patterns': ['Thanks', 'Thank you', "That's helpful", 'Nice', 'Good'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure']},
  {'tag': 'about',
   'patterns': ['about you',
    'Who are you?',
    'What are you?',
    'Who you are?',
    'what you can do',
    'What is the Purpose of the project'],
   'responses': ['I am AyurBot. I can help determine your prakriti type. Prakriti is our inherent nature, with which we are born. It is the nature of our body in terms of dosha. It dies not change duri

In [289]:
for intent in intents["intent"]:
    for pattern in intent["patterns"]:
        # separate each word from the pattern
        w=nltk.word_tokenize(pattern)
        # add it in the words list
        words.extend(w)
        
        # adding it in the documents with the defined tag.
        documents.append((w,intent["tag"]))

        # adding the classes in the classes list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [290]:
words

['Hi',
 'How',
 'are',
 'you',
 '?',
 'Hey',
 'there',
 ',',
 'how',
 'have',
 'you',
 'been',
 '?',
 'Is',
 'anyone',
 'there',
 '?',
 'Hello',
 'Good',
 'day',
 'Hiii',
 'Hii',
 'Bye',
 'See',
 'you',
 'there',
 'Goodbye',
 'Thanks',
 'Thank',
 'you',
 'That',
 "'s",
 'helpful',
 'Nice',
 'Good',
 'about',
 'you',
 'Who',
 'are',
 'you',
 '?',
 'What',
 'are',
 'you',
 '?',
 'Who',
 'you',
 'are',
 '?',
 'what',
 'you',
 'can',
 'do',
 'What',
 'is',
 'the',
 'Purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'I',
 'call',
 'you',
 'whats',
 'your',
 'name',
 '?',
 'Could',
 'you',
 'help',
 'me',
 '?',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'Can',
 'you',
 'help',
 '?',
 'What',
 'can',
 'you',
 'do',
 'for',
 'me',
 '?',
 'I',
 'need',
 'a',
 'support',
 'I',
 'need',
 'a',
 'help',
 'support',
 'me',
 'please',
 'Complete',
 'the',
 'work',
 'My',
 'name',
 'user',
 'name',
 'you',
 'are',
 'not',
 'Good',
 'Nothing',
 'You',
 'are',
 

In [291]:
for doc in documents:
    print(doc)

(['Hi'], 'greeting')
(['How', 'are', 'you', '?'], 'greeting')
(['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting')
(['Is', 'anyone', 'there', '?'], 'greeting')
(['Hello'], 'greeting')
(['Good', 'day'], 'greeting')
(['Hiii'], 'greeting')
(['Hii'], 'greeting')
(['Bye'], 'goodbye')
(['See', 'you', 'there'], 'goodbye')
(['Goodbye'], 'goodbye')
(['Thanks'], 'thanks')
(['Thank', 'you'], 'thanks')
(['That', "'s", 'helpful'], 'thanks')
(['Nice'], 'thanks')
(['Good'], 'thanks')
(['about', 'you'], 'about')
(['Who', 'are', 'you', '?'], 'about')
(['What', 'are', 'you', '?'], 'about')
(['Who', 'you', 'are', '?'], 'about')
(['what', 'you', 'can', 'do'], 'about')
(['What', 'is', 'the', 'Purpose', 'of', 'the', 'project'], 'about')
(['what', 'is', 'your', 'name'], 'name')
(['what', 'should', 'I', 'call', 'you'], 'name')
(['whats', 'your', 'name', '?'], 'name')
(['Could', 'you', 'help', 'me', '?'], 'help')
(['give', 'me', 'a', 'hand', 'please'], 'help')
(['Can', 'you', 'help', '?'], 'h

In [292]:
classes

['greeting',
 'goodbye',
 'thanks',
 'about',
 'name',
 'help',
 'noanswer',
 'prakriti',
 'creators',
 'badwords',
 'acknowledge',
 'age_inquiry',
 'positive_feedback',
 'negative_feedback',
 'information_request',
 'contact_support',
 'diet_recommendation',
 'exercise_suggestions',
 'stress_management',
 'sleep_tips',
 'nutrition_information',
 'exit']

Lemmatization on words and removing the words which are not usable

In [293]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words

['hi',
 'how',
 'are',
 'you',
 'hey',
 'there',
 ',',
 'how',
 'have',
 'you',
 'been',
 'is',
 'anyone',
 'there',
 'hello',
 'good',
 'day',
 'hiii',
 'hii',
 'bye',
 'see',
 'you',
 'there',
 'goodbye',
 'thanks',
 'thank',
 'you',
 'that',
 "'s",
 'helpful',
 'nice',
 'good',
 'about',
 'you',
 'who',
 'are',
 'you',
 'what',
 'are',
 'you',
 'who',
 'you',
 'are',
 'what',
 'you',
 'can',
 'do',
 'what',
 'is',
 'the',
 'purpose',
 'of',
 'the',
 'project',
 'what',
 'is',
 'your',
 'name',
 'what',
 'should',
 'i',
 'call',
 'you',
 'whats',
 'your',
 'name',
 'could',
 'you',
 'help',
 'me',
 'give',
 'me',
 'a',
 'hand',
 'please',
 'can',
 'you',
 'help',
 'what',
 'can',
 'you',
 'do',
 'for',
 'me',
 'i',
 'need',
 'a',
 'support',
 'i',
 'need',
 'a',
 'help',
 'support',
 'me',
 'please',
 'complete',
 'the',
 'work',
 'my',
 'name',
 'user',
 'name',
 'you',
 'are',
 'not',
 'good',
 'nothing',
 'you',
 'are',
 'bad',
 'noting',
 'to',
 'worry',
 'chatgpt',
 'is',
 'best

Sorting the words

In [294]:
words = sorted(list(set(words)))
words

["'m",
 "'re",
 "'s",
 ',',
 'a',
 'about',
 'age',
 'annoying',
 'anyone',
 'are',
 'away',
 'bad',
 'be',
 'been',
 'best',
 'by',
 'bye',
 'ca',
 'call',
 'can',
 'chatbot',
 'chatgpt',
 'complete',
 'contact',
 'could',
 'created',
 'creator',
 'customer',
 'day',
 'despise',
 'detail',
 'determine',
 'diet',
 'disappointed',
 'do',
 'doing',
 'eat',
 'eating',
 'exercise',
 'exit',
 'fact',
 'feeling',
 'figure',
 'find',
 'food',
 'for',
 'fuck',
 'get',
 'give',
 'go',
 'good',
 'goodbye',
 'great',
 'hand',
 'hate',
 'have',
 'healthy',
 'hello',
 'help',
 'helpful',
 'hey',
 'hi',
 'hii',
 'hiii',
 'how',
 'i',
 'identify',
 'impressed',
 'improve',
 'in',
 'information',
 'irrelevant',
 'is',
 'job',
 'learn',
 'love',
 'made',
 'make',
 'manage',
 'me',
 'more',
 'my',
 "n't",
 'name',
 'need',
 'nice',
 'not',
 'nothing',
 'noting',
 'now',
 'nutrition',
 'of',
 'ok',
 'old',
 'out',
 'over',
 'plan',
 'please',
 'prakriti',
 'project',
 'provide',
 'purpose',
 'quit',
 're

Sorting the Classes

In [295]:
classes = sorted(list(set(classes)))
classes

['about',
 'acknowledge',
 'age_inquiry',
 'badwords',
 'contact_support',
 'creators',
 'diet_recommendation',
 'exercise_suggestions',
 'exit',
 'goodbye',
 'greeting',
 'help',
 'information_request',
 'name',
 'negative_feedback',
 'noanswer',
 'nutrition_information',
 'positive_feedback',
 'prakriti',
 'sleep_tips',
 'stress_management',
 'thanks']

Classes and words and Documents overview

In [296]:
print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)

101 documents
22 classes ['about', 'acknowledge', 'age_inquiry', 'badwords', 'contact_support', 'creators', 'diet_recommendation', 'exercise_suggestions', 'exit', 'goodbye', 'greeting', 'help', 'information_request', 'name', 'negative_feedback', 'noanswer', 'nutrition_information', 'positive_feedback', 'prakriti', 'sleep_tips', 'stress_management', 'thanks']
149 unique lemmatized words ["'m", "'re", "'s", ',', 'a', 'about', 'age', 'annoying', 'anyone', 'are', 'away', 'bad', 'be', 'been', 'best', 'by', 'bye', 'ca', 'call', 'can', 'chatbot', 'chatgpt', 'complete', 'contact', 'could', 'created', 'creator', 'customer', 'day', 'despise', 'detail', 'determine', 'diet', 'disappointed', 'do', 'doing', 'eat', 'eating', 'exercise', 'exit', 'fact', 'feeling', 'figure', 'find', 'food', 'for', 'fuck', 'get', 'give', 'go', 'good', 'goodbye', 'great', 'hand', 'hate', 'have', 'healthy', 'hello', 'help', 'helpful', 'hey', 'hi', 'hii', 'hiii', 'how', 'i', 'identify', 'impressed', 'improve', 'in', 'infor

In [297]:
documents

[(['Hi'], 'greeting'),
 (['How', 'are', 'you', '?'], 'greeting'),
 (['Hey', 'there', ',', 'how', 'have', 'you', 'been', '?'], 'greeting'),
 (['Is', 'anyone', 'there', '?'], 'greeting'),
 (['Hello'], 'greeting'),
 (['Good', 'day'], 'greeting'),
 (['Hiii'], 'greeting'),
 (['Hii'], 'greeting'),
 (['Bye'], 'goodbye'),
 (['See', 'you', 'there'], 'goodbye'),
 (['Goodbye'], 'goodbye'),
 (['Thanks'], 'thanks'),
 (['Thank', 'you'], 'thanks'),
 (['That', "'s", 'helpful'], 'thanks'),
 (['Nice'], 'thanks'),
 (['Good'], 'thanks'),
 (['about', 'you'], 'about'),
 (['Who', 'are', 'you', '?'], 'about'),
 (['What', 'are', 'you', '?'], 'about'),
 (['Who', 'you', 'are', '?'], 'about'),
 (['what', 'you', 'can', 'do'], 'about'),
 (['What', 'is', 'the', 'Purpose', 'of', 'the', 'project'], 'about'),
 (['what', 'is', 'your', 'name'], 'name'),
 (['what', 'should', 'I', 'call', 'you'], 'name'),
 (['whats', 'your', 'name', '?'], 'name'),
 (['Could', 'you', 'help', 'me', '?'], 'help'),
 (['give', 'me', 'a', 'hand'

dumping the words list and classes list

In [298]:
pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))

Training initializer

In [299]:
training=[]

In [300]:
output_empty=[0]*len(classes)
output_empty

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [301]:
for doc in documents:
    bag=[]
    # extracting the patterns
    pattern_words=doc[0]
    
    # lemmatizing the words from the pattern
    pattern_words=[lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
     # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row=list(output_empty)
    output_row[classes.index(doc[1])]=1
    
    training.append([bag,output_row])
    
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Shuffling our features and turn into np.array

In [302]:
print(training[0][0])

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [303]:
print(training[0])

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]


In [304]:
for train in training:
    print(train)

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
[[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1

In [305]:
training_data=np.array(training,dtype=object)
training_data

array([[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
        list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])],
       [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [306]:
training_data.shape

(101, 2)

Training X and Training Y data

In [307]:
train_x=list(training_data[:,0])
train_x

[[0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,

In [308]:
train_y=list(training_data[:,1])
train_y

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

Creating the Model

In [309]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))

Summary of the model

In [310]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 128)               19200     
                                                                 
 dropout_10 (Dropout)        (None, 128)               0         
                                                                 
 dense_16 (Dense)            (None, 64)                8256      
                                                                 
 dropout_11 (Dropout)        (None, 64)                0         
                                                                 
 dense_17 (Dense)            (None, 22)                1430      
                                                                 
Total params: 28886 (112.84 KB)
Trainable params: 28886 (112.84 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model

In [311]:
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])

In [312]:
model.fit(np.array(train_x),np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200


21/21 [==============================] - 1s 3ms/step - loss: 3.1050 - accuracy: 0.0297 
Epoch 2/200
21/21 [==============================] - 0s 6ms/step - loss: 3.0124 - accuracy: 0.1188
Epoch 3/200
21/21 [==============================] - 0s 3ms/step - loss: 2.9597 - accuracy: 0.1386
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 2.8832 - accuracy: 0.1287
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 2.8299 - accuracy: 0.1584
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 2.6981 - accuracy: 0.2673
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 2.6502 - accuracy: 0.2475
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 2.4964 - accuracy: 0.3168
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 2.4845 - accuracy: 0.2871
Epoch 10/200
21/21 [==============================] - 0s 2ms/step - loss: 2.3398 - accuracy: 0.3366
Epoch 11/200
21/21 [

In [313]:
np.array(train_x)

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

The Model is Trained

Lets predict it using the model

In [314]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [315]:
clean_up_sentence("hello who are you")

['hello', 'who', 'are', 'you']

In [316]:
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


In [317]:
bow("hello who are you",words)

found in bag: hello
found in bag: who
found in bag: are
found in bag: you


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0])

In [318]:
words[9]

'are'

In [319]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    print(res)
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    print(results)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


In [320]:
predict_class("stupid bot",model)

1/1 [==============================] - 0s 88ms/step
[4.3522360e-04 1.8218605e-02 1.3602814e-03 2.8374286e-03 1.2208372e-03
 1.3293243e-03 7.0492207e-04 2.8457523e-03 3.4351446e-02 1.2752201e-02
 1.0646458e-01 2.7746722e-04 4.5984196e-03 1.1148979e-03 3.0681647e-03
 4.5331935e-03 2.7226185e-04 7.0949143e-04 2.7504894e-03 5.7162665e-04
 2.4012183e-03 7.9718220e-01]
[[21, 0.7971822]]


[{'intent': 'thanks', 'probability': '0.7971822'}]

In [321]:
classes

['about',
 'acknowledge',
 'age_inquiry',
 'badwords',
 'contact_support',
 'creators',
 'diet_recommendation',
 'exercise_suggestions',
 'exit',
 'goodbye',
 'greeting',
 'help',
 'information_request',
 'name',
 'negative_feedback',
 'noanswer',
 'nutrition_information',
 'positive_feedback',
 'prakriti',
 'sleep_tips',
 'stress_management',
 'thanks']

In [322]:
def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intent"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result

In [323]:
getResponse(predict_class("prakriti bot",model),intents)

1/1 [==============================] - 0s 32ms/step
[2.7987894e-06 1.3612705e-05 1.9065218e-06 3.2705309e-08 2.2129397e-08
 9.1785921e-08 9.9088618e-07 2.0617778e-08 6.6207360e-07 9.0557442e-06
 1.0760225e-06 2.3677925e-08 2.2686929e-06 1.0370150e-09 4.2296433e-09
 2.6069233e-06 3.9663769e-06 2.6768346e-08 9.9935251e-01 1.0435222e-07
 1.6523482e-07 6.0807139e-04]
[[18, 0.9993525]]


'Absolutely, just need you to answer a few questions. Answer yes to get started.'